# Network motifs
In this notebook, we will be exploring how to work with network motifs. You will first need the following two modules. Networx is one of the main network science libraries for python. It has many functions built in for network analysis https://networkx.org/. We will also need the packages below. You may already have them installed, if so, you can skip the first cell, and just load them in the second cell.

In [ ]:
import sys
!{sys.executable} -m pip install tabulate
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install networkx
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install random

In [ ]:
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
import random as rd
from tabulate import tabulate

We will investigate motifs in the network of network science collaborations, so first let's load the data that you are interested in. Here, I included two sources of data: one of the european road network, and one collaboration network of network scientist. Choose whichever you like, and comment the other one out. 

###  <font color='red'>When using Jupyter notebook on donwloaded GitHub folder, run the cell below</font>

In [ ]:
G = nx.read_adjlist("netsci_data.txt")     #read adjacency list into a graph object
G = nx.read_adjlist("euroroad.txt")     #read adjacency list into a graph object

print('Number of nodes', len(G.nodes))
print('Number of edges', len(G.edges))
avgdeg = sum(dict(G.degree).values()) / len(G.nodes)
print('Average degree', avgdeg)


### <font color='red'>When using Google collab run the two cells below instead:</font>

The rest of the notebook is the same regardless of which method you used. Again, to change to another data set, comment the one that you do not use out.

In [ ]:
!git clone https://github.com/clarastegehuis/Lake_como_school/

In [ ]:
G = nx.read_adjlist('/content/Lake_como_school/euroroad.txt')     #read adjacency list into a graph object
G = nx.read_adjlist('/content/Lake_como_school/netsci_data.txt')     #read adjacency list into a graph object

print('Number of nodes', len(G.nodes))
print('Number of edges', len(G.edges))
avgdeg = sum(dict(G.degree).values()) / len(G.nodes)
print('Average degree', avgdeg)

## Exercise 1 
Count the number of triangles in the data sets

In [ ]:
triad_count = sum(nx.triangles(G).values()) / 3 #In networkx, every triangle is counted 3 times, due to reordering of the vertices
print(triad_count, " triangles")

Now let's say that we take an Erdos Renyi model as a null model to test for whether a statistically significant number of triangles is present. First figure out what the correct values of $n$ and $p$ are to compare the ER-model with your data set.

In [ ]:
n =                # fill in the correct value of n to compare to the data
p =      # fill in the correct value of p to compare to the data
print('p = ', p, ' n = ', n)

With these values for $n$ and $p$, we can generate let's say 100 ER models that should serve as null models for the data sets, and count triangles in those null models. 

In [ ]:
test_triad_counts = []  # vector with triangle counts in the ER model
for i in range(100):
    G_test = nx.erdos_renyi_graph(n,p)
    test_triad_counts.append(sum(nx.triangles(G_test).values()) / 3)

Now plot a histogram of the generated triangle counts from the ER model together with the triangle count in the network data

In [ ]:
def plot_triads(countvec,datacount):    #This function plots a histogram of countvec, with a horizontal line at datacount
    fig, ax = plt.subplots(figsize =(10, 7))
    ax.hist(countvec, alpha = 0.65)
    plt.style.use("bmh")

    ax.set_yticks([])
    ax.grid(False)
    ax.tick_params(left = False, bottom = False)
    for ax, spine in ax.spines.items():
        spine.set_visible(False)
    plt.xlabel("triangle counts",size = 17)
    plt.ylabel("frequency",size = 17)
    plt.axvline(datacount, color = 'k', linestyle='dashed', linewidth=2)

    # Show plot
    plt.show()

In [ ]:
plot_triads(test_triad_counts,triad_count)

Now compute the Z-score of the triangle counts:

In [ ]:
def Zscore(countvec,datacount):       #compute the Z score of datacount, when the null model produces as samples countvec
    meancount = np.average(countvec)
    std_count = np.std(countvec)     # standard deviation of triangle counts in null model
    return (datacount-meancount)/std_count

In [ ]:
print('Zscore is ',Zscore(test_triad_counts,triad_count))

### Configuration model
Now let's try the same thing in the configuration model. The input for the configuration model is a degree sequence, and it creates a random graph on those degrees. So first generate the correct input for the configuration model to be able to compare it to the data, and then compare triangle counts. 

In [ ]:
degree_sequence =  # Pick the correct degree sequence to compare to the network data

test_triad_config = []
for i in range(100):
    G_config = nx.configuration_model(degree_sequence)
    G_config= nx.Graph(G_config)
    test_triad_config.append(sum(nx.triangles(G_config).values()) / 3)
plot_triads(test_triad_config,triad_count)
print('Zscore is ',Zscore(test_triad_config,triad_count))

### Geometric random graph 
Now let's try yet another null model, the geometric random graph. Here, the input parameters are the radius $r$ and the number of vertices $n$. This model assumes that all $n$ vertices are uniformly placed in a $[0,1]^2$ box, and that every vertex connects with vertices within radius $r$. First, find the values of $r$ and $n$ for which you would be able to compare them to your data set, and then compare triangle counts. You may need that np.pi is the number $\pi$.

In [ ]:
radius =  np.sqrt(avgdeg/(n*np.pi)) # compute the radius which makes the Geometrc random graph a good null model for the data

G_geom = nx.random_geometric_graph(n, radius)

test_triad_geom = []
for i in range(100):
    G_geom = nx.random_geometric_graph(n, radius)
    test_triad_geom.append(sum(nx.triangles(G_geom).values()) / 3)
plot_triads(test_triad_geom,triad_count)
print('Zscore is ', Zscore(test_triad_geom,triad_count))

## Questions
What are the differences between the different null models?

Are triangles always significant according to the Z-score?

Are there differences when selecting different data sets?

What is a good null model for these data?

# Part 2
##  Other motifs
We now investigate some other motif types. For simplicity, we will stick with cliques. First, lets generate a list of all cliques in the network, and a function that then counts all cliques of size $k$.

In [ ]:
clique_list = nx.enumerate_all_cliques(G)   

In [ ]:
def cliquenumber(cliques,k):                #A function that takes a list with all cliques in the graph, and then reports the number of k-cliques
    k_clique_num = 0
    for i in cliques:
        if len(i)==k:
            k_clique_num += 1
    return k_clique_num

Pick a value of $k$ that you are interested in (probably very large cliques do not appear so often, so you may want to choose it not so much bigger than 3). Then perform the clique count in the data.

In [ ]:
k = 3
clique_count_G = cliquenumber(clique_list,k)
print(clique_count_G)

You can now try the different null models for larger clique sizes and compare. The code now has the ER graph as a null mode, but you can try the other two null models as well:

In [ ]:
test_kclique_counts = []
for i in range(100):
    G_test = nx.erdos_renyi_graph(n,p)
    G_test_cliques = nx.enumerate_all_cliques(G_test) 
    test_kclique_counts.append(cliquenumber(G_test_cliques,k))
plot_triads(test_kclique_counts,clique_count_G)

### Questions
Do other $k$-cliques appear significantly often compared to the null models?

# Part 3
## Sampling
Sometimes it is not possible to count the exact number of subgraphs in a large networks due to large computation times. One of the solutions to this problem is to use sampling instead. Many methods to sample subgraphs exist, but here we focus on the simplest one of them, creating a subsample of the network where every node is included with probability $q$. Then, it computes subgraphs (in this case cliques) in the reduced graph. Figure out how to obtain an estimate for the number of cliques in the larger graph from this sampled clique count. 

In [ ]:
def sampled_cliques(Graph,sample_prob,size): # This function takes in a graph Graph, and returns an estimated clique count for cliques of size 'size'
    nsampled = int((1-sample_prob)*len(Graph.nodes()))
    sample = rd.sample(list(Graph.nodes), nsampled)    
    H=Graph.copy()
    H.remove_nodes_from(sample)            #remove every vertex in Graph independently with probability 1-p

    clique_list_sampled = nx.enumerate_all_cliques(H)  
    sampled_clique_count = cliquenumber(clique_list_sampled,size)        # the clique count in the subsampled graph

    estimated_clique_count =            # figure out how to compute the estimated clique count from sampled_clique_count
    return estimated_clique_count

The cell below creates 100 subgraph count estimates with sampling probability $q$, and plots them together with the data count (dashed line). 

In [ ]:
q = 0.7
sampledvec = []
for i in range(100):
    sampledvec.append(sampled_cliques(G,q,k))
plot_triads(sampledvec,clique_count_G)

### Questions
How close is the sampled subgraph count to the actual one?

How small can $q$ be to still obtain accurate estimates?

How does this depend on the clique size $k$?

# Part 4: Neighborhood sampling
Below, we would like to create an unbiased neighborhood sampling algorithm to approximate the density of triangles. First, we define a function that computes the triangle density of a graph:

In [ ]:
def triad_density(Graph):
    sumpairs = 0
    degrees = [d for n, d in G.degree()]
    for d in degrees:
        sumpairs += d*(d-1)/2
    sumpairs -= 2* triad_count
    return triad_count/sumpairs

In [ ]:
print(triad_density(G))

Now, we create a neighborhood sampling algorithm to approximate the triangle density. Add the probabilities that the algorithm samples the paths and subgraphs below to complete it, and compare it to the empirical triad density.

In [ ]:
G_edge_count = len(G.edges())
S_triad=0                                                   #sum of weights for triangles
S_tot=0                                                     #sum of weights for all 3-nodes subgraphs
N_samples = 500                                             # The number of random samples
for k in range(N_samples):                                        
    random_sampled_edge = rd.sample(list(G.edges),1)        #sample random edge (u,v)
    u = random_sampled_edge[0][0]
    v = random_sampled_edge[0][1]
    sample_list = []
    for i in G[u]:
        if i!=v:
            sample_list.append(i)                           #add all neighbors of u to the list
    for j in G[v]:
        if j!=u:
            sample_list.append(j)                           #add all neighbors of v to the list
    if sample_list == []:
        continue
    w = rd.sample(sample_list,1)[0]                         #sample random vertex w  from the list
    
    if G.has_edge(u,w) and G.has_edge(v,w):                 # if u,v,w is a triangle
        P=   #What is the probability that triangle (u,v,w) is sampled? You may use that the number of neighbors of a node u is len(G[u])
        S_triad+=1/P
    else:
        if G.has_edge(u,w):
            P=         #What is the probability that path (w,u,v) is sampled?
        if G.has_edge(v,w):
            P=         #What is the probability that path (u,v,w) is sampled?
            
    S_tot+=1/P
print(S_triad/S_tot)

# Part 5

Instead of sampling or exact computation of the subgraph counts in your null model, it is sometimes also possible to obtain analytical estimates for them. This can get quite involved depending on the exact null models you use, but for the Erdos Renyi model this is still doable. You may use that
$$\text{Var}(N(\triangle)) = \sum_{i,j,k,s,t,u}P(i,j,k \text{ and }{s,t,u}\text{ triangles})-p^6 $$
Now compare the mean and variance of the triangle counts you found experimentally with the mean and variance you computed analytically. Are they similar?

In [ ]:
theoretical_triads_mean =        # Compute the mean number of triangles
theoretical_var =         # Compute the variance of the number of triangles

experimental_mean = np.mean(test_triad_counts)              # Compute the mean number of triangles in the sampled ER graphs
experimental_var = np.var(test_triad_counts)

table = [[' ', 'Mean', 'Variance'], ['Experimental', experimental_mean,experimental_var ],['Theoretical', theoretical_triads_mean,theoretical_var ]]

print(tabulate(table))

Compare the theoretical mean number of larger $k$ cliques as well to its sampled value

In [ ]:
theoretical_cliques_mean =            # Compute the mean number of k-cliques
experimental_mean = np.mean(test_kclique_counts) 

print('Theoretical mean ',theoretical_cliques_mean, 'Experimental mean ', experimental_mean)